<a href="https://colab.research.google.com/github/AHMerrill/unstructured-project/blob/main/anti_echo_chamber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# test cell

# ================================================================
# Anti Echo Chamber — Full Analysis and Retrieval Pipeline
# ================================================================

This notebook performs a complete, production-grade anti-echo workflow:

1. Secure OpenAI login  
2. Rebuild ChromaDB from Hugging Face  
3. Upload and parse PDF / TXT / HTML  
4. Summarize with OpenAI (`gpt-4o-mini`)  
5. Create topic + stance embeddings  
6. Compare against Chroma to surface ideologically contrasting articles  

Repositories  
- GitHub: https://github.com/AHMerrill/anti-echo-chamber  
- Hugging Face dataset: https://huggingface.co/datasets/zanimal/anti-echo-artifacts


In [2]:
# ================================================================
# Stage 1 — Secure OpenAI API Key Setup
# ================================================================

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"]:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

print("OpenAI API key loaded into environment (hidden)")


Enter your OpenAI API key: ··········
OpenAI API key loaded into environment (hidden)


# ================================================================
# Stage 2 — Environment Setup and Repository Configuration
# ================================================================

This stage:
- Clones the GitHub repo
- Installs dependencies
- Loads YAML + JSON configs
- Prints the active models and Chroma settings


In [3]:
# ================================================================
# Stage 2 — Environment Setup and Repository Configuration (config-driven)
# ================================================================

import os, json, yaml
from pathlib import Path

# --- Paths ---
PROJECT_ROOT = Path("/content/unstructured-project").resolve()

# --- Clone correct repo if missing ---
if not PROJECT_ROOT.exists():
    print("Cloning repo from GitHub...")
    os.system("git clone https://github.com/AHMerrill/unstructured-project.git /content/unstructured-project")
else:
    print("Repository exists. Pulling latest changes...")
    os.system(f"cd {PROJECT_ROOT} && git pull")

# --- Load config.yaml from correct path ---
CONFIG_PATH = PROJECT_ROOT / "config/config.yaml"
if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"Config file not found at: {CONFIG_PATH}")

with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    CONFIG = yaml.safe_load(f)

# --- Install dependencies ---
!pip install -q pdfplumber beautifulsoup4 chromadb sentence-transformers pyyaml huggingface_hub openai rapidfuzz

summary = {
    "repo": "AHMerrill/unstructured-project",
    "hf_dataset_id": CONFIG.get("hf_dataset_id"),
    "topic_model": CONFIG.get("embeddings", {}).get("topic_model"),
    "stance_model": CONFIG.get("embeddings", {}).get("stance_model"),
    "chroma_collections": CONFIG.get("chroma_collections"),
}
print(json.dumps(summary, indent=2))


Cloning repo from GitHub...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 128.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17

# ================================================================
# Stage 3 — Full Chroma Rebuild from Hugging Face Dataset
# ================================================================

This stage reconstructs the local **ChromaDB** from your Hugging Face dataset  
[`zanimal/anti-echo-artifacts`](https://huggingface.co/datasets/zanimal/anti-echo-artifacts).

It preserves the full multi-topic and multi-stance structure from your scraper:
- Each article can yield **multiple topic vectors** (`::topic::0`, `::topic::1`, …)
- Each article can yield **multiple stance vectors** (`::stance::summary`, `::stance::0`, …)

Duplicates are filtered **only by exact row_id**, not by base article ID.  
This ensures we retain all topical clusters while preventing re-ingestion of the same batch.


In [4]:
# ================================================================
# Stage 3 — Full Chroma Rebuild from Hugging Face Dataset (config-driven)
# ================================================================

import os, json, numpy as np, traceback
from pathlib import Path
from huggingface_hub import list_repo_files, hf_hub_download
import chromadb
from collections import defaultdict

PROJECT_ROOT = Path("/content/unstructured-project").resolve()
CHROMA_PATH  = PROJECT_ROOT / "chroma_db"
CHROMA_PATH.mkdir(parents=True, exist_ok=True)

HF_REPO = CONFIG["hf_dataset_id"]
topic_name  = CONFIG["chroma_collections"]["topic"]
stance_name = CONFIG["chroma_collections"]["stance"]

client = chromadb.PersistentClient(path=str(CHROMA_PATH))

# Drop and recreate clean collections
for name in [topic_name, stance_name]:
    try:
        client.delete_collection(name)
    except Exception:
        pass
topic_coll  = client.create_collection(topic_name,  metadata={"hnsw:space": "cosine"})
stance_coll = client.create_collection(stance_name, metadata={"hnsw:space": "cosine"})
print(f"Initialized collections '{topic_name}' and '{stance_name}' at {CHROMA_PATH}")

# --- Helpers (unchanged) ---
def load_npz_safely(path):
    arr = np.load(path, allow_pickle=False)
    if isinstance(arr, np.lib.npyio.NpzFile):
        for key in arr.files:
            if arr[key].ndim == 2:
                return arr[key]
        raise ValueError(f"No 2D arrays found in {path}")
    return arr

def load_jsonl(fp):
    records = []
    with open(fp, "r", encoding="utf-8") as f:
        for line in f:
            line=line.strip()
            if not line: continue
            try: records.append(json.loads(line))
            except: continue
    return records

# --- Discover batches ---
files = list_repo_files(HF_REPO, repo_type="dataset")
batches = sorted({"/".join(f.split("/")[:2]) for f in files if f.startswith("batches/")})
print(f"Detected {len(batches)} batches in {HF_REPO}")

# --- Rebuild loop (same logic as before) ---
seen_topic_ids, seen_stance_ids = set(), set()
article_topic_counts, article_stance_counts = defaultdict(int), defaultdict(int)
topic_total = stance_total = 0

for batch in batches:
    try:
        print(f"\n--- Processing {batch} ---")
        topic_npz  = hf_hub_download(HF_REPO, f"{batch}/embeddings_topic.npz",  repo_type="dataset")
        stance_npz = hf_hub_download(HF_REPO, f"{batch}/embeddings_stance.npz", repo_type="dataset")
        meta_topic = hf_hub_download(HF_REPO, f"{batch}/metadata_topic.jsonl",  repo_type="dataset")
        meta_stance= hf_hub_download(HF_REPO, f"{batch}/metadata_stance.jsonl", repo_type="dataset")

        t_embs, s_embs = load_npz_safely(topic_npz), load_npz_safely(stance_npz)
        t_meta, s_meta = load_jsonl(meta_topic),  load_jsonl(meta_stance)

        # topic upsert
        t_records=[]
        for e,m in zip(t_embs,t_meta):
            rid=m.get("row_id") or f"{m.get('id','unknown')}::topic::0"
            if rid in seen_topic_ids: continue
            seen_topic_ids.add(rid); t_records.append((rid,e,m))
            article_topic_counts[rid.split('::')[0]]+=1
        if t_records:
            topic_coll.upsert(
                ids=[r[0] for r in t_records],
                embeddings=[r[1].tolist() for r in t_records],
                metadatas=[r[2] for r in t_records])
        topic_total+=len(t_records)

        # stance upsert
        s_records=[]
        for e,m in zip(s_embs,s_meta):
            rid=m.get("row_id") or f"{m.get('id','unknown')}::stance::0"
            if rid in seen_stance_ids: continue
            seen_stance_ids.add(rid); s_records.append((rid,e,m))
            article_stance_counts[rid.split('::')[0]]+=1
        if s_records:
            stance_coll.upsert(
                ids=[r[0] for r in s_records],
                embeddings=[r[1].tolist() for r in s_records],
                metadatas=[r[2] for r in s_records])
        stance_total+=len(s_records)
        print(f"✓ Added {len(t_records)} topic, {len(s_records)} stance vectors")
    except Exception as e:
        print(f"Failed {batch}: {type(e).__name__}: {e}")
        traceback.print_exc(limit=1)

print("\n=== Rebuild Summary ===")
print(f"Topic vectors added: {topic_total}")
print(f"Stance vectors added: {stance_total}")
print(f"Stored at {CHROMA_PATH}")


Initialized collections 'unstructured_topic' and 'unstructured_stance' at /content/unstructured-project/chroma_db


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Detected 2 batches in zanimal/unstructured-project

--- Processing batches/batch_20251025T173713Z_5dd9b9dd ---


batches/batch_20251025T173713Z_5dd9b9dd/(…):   0%|          | 0.00/89.9k [00:00<?, ?B/s]

batches/batch_20251025T173713Z_5dd9b9dd/(…):   0%|          | 0.00/18.7k [00:00<?, ?B/s]

...

batches/batch_20251025T181241Z_d8804259/(…):   0%|          | 0.00/3.04M [00:00<?, ?B/s]

batches/batch_20251025T181241Z_d8804259/(…):   0%|          | 0.00/614k [00:00<?, ?B/s]

metadata_topic.jsonl: 0.00B [00:00, ?B/s]

metadata_stance.jsonl: 0.00B [00:00, ?B/s]

✓ Added 2104 topic, 419 stance vectors

=== Rebuild Summary ===
Topic vectors added: 2168
Stance vectors added: 432
Stored at /content/unstructured-project/chroma_db


# ================================================================
# Stage 4 — User Upload + Source Bias Detection
# ================================================================

This stage handles ingestion of a user-supplied article (TXT / PDF / HTML).  
It performs four key steps:

1. **Upload** the file and extract plain text.  
2. **Infer or confirm** the publication source.  
3. **Match** against existing entries in `source_bias.json` (fuzzy).  
4. **If new**, infer ideological metadata (bias family + score) using `gpt-4o-mini`.

All extracted and inferred data will be cached for later topic + stance analysis.


In [11]:
# ================================================================
# Stage 4 — User Upload + Source Bias Detection
# ================================================================

import os, re, json, pdfplumber, requests
from bs4 import BeautifulSoup
from rapidfuzz import process, fuzz
from pathlib import Path
from openai import OpenAI
from getpass import getpass

# --- Environment setup ---
PROJECT_ROOT = Path("/content/unstructured-project").resolve()
CONFIG_DIR = PROJECT_ROOT / "config"

# Load your bias map
SOURCE_BIAS_PATH = CONFIG_DIR / "source_bias.json"
SOURCE_BIAS = json.load(open(SOURCE_BIAS_PATH, encoding="utf-8"))

# Ensure OpenAI key
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"].strip():
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# --------------------------------------------------------------------
# 1. File Upload + Text Extraction
# --------------------------------------------------------------------
from google.colab import files
import io

uploaded = files.upload()
filename = list(uploaded.keys())[0]
file_ext = Path(filename).suffix.lower()

def extract_text(path):
    if path.endswith(".txt"):
        return Path(path).read_text(encoding="utf-8", errors="ignore")
    if path.endswith(".pdf"):
        text = ""
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text += page.extract_text() or ""
        return text
    if path.endswith(".html") or path.endswith(".htm"):
        soup = BeautifulSoup(Path(path).read_text(encoding="utf-8", errors="ignore"), "html.parser")
        for s in soup(["script", "style"]):
            s.decompose()
        return soup.get_text(separator=" ")
    raise ValueError("Unsupported file type")

article_text = extract_text(filename).strip()
print(f"Extracted {len(article_text)} characters from {filename}")

# --------------------------------------------------------------------
# 2. Attempt Source Inference
# --------------------------------------------------------------------
def infer_source_name(text):
    # Quick domain sniff
    m = re.search(r"https?://([^/\s]+)", text)
    if m:
        domain = m.group(1).lower()
        domain = domain.replace("www.", "")
        return domain.split(".")[0]
    # GPT fallback
    prompt = (
        "You are a media analyst. Based on the article text below, "
        "infer the most likely publication or outlet name. "
        "Return only the outlet name, e.g., 'The Guardian', 'Fox News', etc.\n\n"
        f"Article excerpt:\n{text[:2000]}"
    )
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=20,
        temperature=0.2
    )
    guess = resp.choices[0].message.content.strip()
    return re.sub(r"[^A-Za-z0-9\s\-]", "", guess)

inferred_source = infer_source_name(article_text)
print(f"🕵️ Inferred possible source: {inferred_source}")

# Ask user to confirm or override
user_resp = input(f"Is this article from '{inferred_source}'? [y/n]: ").strip().lower()
if user_resp != "y":
    user_source = input("Enter the source name (may contain typos): ").strip()
    confirmed_source = user_source
else:
    confirmed_source = inferred_source

# --------------------------------------------------------------------
# 3. Fuzzy Match Against Known Sources
# --------------------------------------------------------------------
known_sources = list(SOURCE_BIAS.keys())
match, score, _ = process.extractOne(confirmed_source, known_sources, scorer=fuzz.ratio)
print(f"Closest match: {match} (score {score})")

if score >= 85:
    bias_info = SOURCE_BIAS[match]
    print(f"Matched existing bias entry for {match}")
else:
    # ----------------------------------------------------------------
    # 4. GPT Bias Inference Fallback  (FIXED JSON PARSING)
    # ----------------------------------------------------------------
    prompt = f"""
You are a media bias researcher.
Given the outlet name "{confirmed_source}", infer its general political bias family
(e.g., 'center left', 'center right', 'libertarian right', 'progressive left', 'neutral').
Return JSON with:
- bias_family
- bias_score (float, -1.0 = far left, +1.0 = far right)
- short_rationale (brief explanation)
"""
    resp = client.chat.completions.create(
        model=CONFIG["stance_processing"]["llm"]["model"],
        messages=[{"role": "user", "content": prompt}],
        max_tokens=128,
        temperature=0.4
    )

    # --- FIXED: safely parse fenced JSON output from GPT ---
    raw_reply = resp.choices[0].message.content.strip()
    clean_json = re.sub(r"```json|```", "", raw_reply).strip()

    try:
        parsed = json.loads(clean_json)
        bias_info = {
            "bias_family": parsed.get("bias_family", "unknown"),
            "bias_score": float(parsed.get("bias_score", 0.0)),
            "short_rationale": parsed.get("short_rationale", "")
        }
    except Exception:
        # fallback: store text if parsing fails
        bias_info = {
            "bias_family": "unknown",
            "bias_score": 0.0,
            "short_rationale": raw_reply
        }

    print(f"New outlet inferred:\n{json.dumps(bias_info, indent=2)}")

# --------------------------------------------------------------------
# 5. Cache Inferred Metadata
# --------------------------------------------------------------------
ARTICLE_META = {
    "filename": filename,
    "source_input": confirmed_source,
    "matched_source": match if score >= 85 else None,
    "bias_family": bias_info.get("bias_family", "unknown"),
    "bias_score": bias_info.get("bias_score", 0.0),
    "rationale": bias_info.get("short_rationale", ""),
    "chars": len(article_text)
}

# Save to temporary workspace
TEMP_DIR = PROJECT_ROOT / "tmp"
TEMP_DIR.mkdir(parents=True, exist_ok=True)
meta_path = TEMP_DIR / f"{Path(filename).stem}_meta.json"
text_path = TEMP_DIR / f"{Path(filename).stem}.txt"
Path(text_path).write_text(article_text, encoding="utf-8")
Path(meta_path).write_text(json.dumps(ARTICLE_META, indent=2), encoding="utf-8")

# --- NEW: Save pointer for Stage 5a ---
latest_ref = TEMP_DIR / "latest_meta.txt"
latest_ref.write_text(str(meta_path))
print(f"\nSaved reference for next stage: {latest_ref}")

# --------------------------------------------------------------------
# Display summary
# --------------------------------------------------------------------
print("\n--- Source Bias Summary ---")
print(json.dumps(ARTICLE_META, indent=2))
print(f"Cached article + metadata under {TEMP_DIR}")


Saving NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press.pdf to NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press (1).pdf


Extracted 8738 characters from NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press (1).pdf
🕵️ Inferred possible source: timesfreepress
Is this article from 'timesfreepress'? [y/n]: n
Enter the source name (may contain typos): nyt opinion
Closest match: wsj-opinion (score 63.63636363636363)
New outlet inferred:
{
  "bias_family": "progressive left",
  "bias_score": -0.7,
  "short_rationale": "The New York Times Opinion section generally promotes progressive viewpoints on social and economic issues, advocating for policies that align with left-leaning ideologies."
}

Saved reference for next stage: /content/unstructured-project/tmp/latest_meta.txt

--- Source Bias Summary ---
{
  "filename": "NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press (1).pdf",
  "source_input": "nyt opinion",
  "matched_source": null,
  "bias_family": "progressive left",
  "bias_score": -0.7,
  "rationale": "The New York Times Opinion section

# ================================================================
# Stage 5a — Topic Embedding with Composite Vector (Base + GPT Summary)
# ================================================================

This step creates a **two-layer topic representation** for precise matching:

### What it does:
1. **Loads** the extracted text and metadata from Stage 4
2. **Generates base topic embeddings** via hierarchical clustering (same as scraper)
3. **Assigns canonical topics** from the predefined taxonomy (e.g., "Politics / Global / Geopolitics & Conflict")
4. **NEW: Generates GPT summary** - A one-sentence description of the article's specific subject (e.g., "Trump imposes sanctions on Russian oil companies amid Ukraine conflict")
5. **Creates composite embedding** by concatenating:
   - Base topic vector (768 dimensions) - captures broad semantic topic
   - GPT summary vector (768 dimensions) - captures specific subject details
   - Final output: 1536-dimensional composite vector

### Output files:
- `{article}_topic_composite.npy` - Composite 1536-dim vector (used for retrieval)
- `{article}_topic.npy` - Base topic vectors (for backward compatibility)
- `{article}_topics_flat.json` - List of canonical topic labels
- `{article}_topic_summary.json` - GPT summary + metadata


In [12]:
# ================================================================
# Stage 5a — Topic Embedding + Canonical Topic Assignment + GPT Summary (Composite)
# ================================================================

import json, numpy as np, nltk, torch, os
from pathlib import Path
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
from getpass import getpass

PROJECT_ROOT = Path("/content/unstructured-project").resolve()
CONFIG_DIR = PROJECT_ROOT / "config"
TMP = PROJECT_ROOT / "tmp"
EPHEMERAL = TMP / "ephemeral_embeddings"
EPHEMERAL.mkdir(parents=True, exist_ok=True)

# --- OpenAI setup ---
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"].strip():
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# --- Load text + meta (FIXED to use explicit reference from Stage 4) ---
latest_meta_ref = TMP / "latest_meta.txt"
if not latest_meta_ref.exists():
    raise FileNotFoundError("Missing latest_meta.txt — run Stage 4 first to upload an article.")

latest_meta = Path(latest_meta_ref.read_text().strip())
if not latest_meta.exists():
    raise FileNotFoundError(f"Referenced meta file not found: {latest_meta}")

meta = json.load(open(latest_meta))
text_path = TMP / f"{Path(latest_meta).stem.replace('_meta','')}.txt"
if not text_path.exists():
    raise FileNotFoundError(f"Text file not found for: {text_path}")

article_text = text_path.read_text(encoding="utf-8")
print(f"Embedding topics for: {meta['filename']} ({len(article_text)} chars)")

# --- Config parameters ---
topic_model_name = CONFIG["embeddings"]["topic_model"]
chunk_tokens = CONFIG["embeddings"]["chunk_tokens"]
normalize = CONFIG["embeddings"]["normalize"]
threshold = CONFIG["topics"]["similarity_threshold"]
max_topics_per_vec = CONFIG["topics"]["max_topics_per_article"]
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using topic model: {topic_model_name}")
tokenizer = AutoTokenizer.from_pretrained(topic_model_name, use_fast=True)
embedder = SentenceTransformer(topic_model_name, device=device)

# --- Load canonical topic anchors + labels ---
anchors_path = CONFIG_DIR / "topic_anchors.npz"
topics_path = CONFIG_DIR / "topics.json"
anchors_npz = np.load(anchors_path, allow_pickle=True)
topic_anchors = {k: anchors_npz[k] for k in anchors_npz.files}
topic_labels = list(topic_anchors.keys())
print(f"Loaded {len(topic_anchors)} topic anchors")

# --- NLTK setup ---
for pkg in ["punkt", "punkt_tab"]:
    try:
        nltk.data.find(f"tokenizers/{pkg}")
    except LookupError:
        nltk.download(pkg)

def sent_split(text):
    return [s.strip() for s in nltk.sent_tokenize(text) if s.strip()]

def encode(texts):
    if isinstance(texts, str):
        texts = [texts]
    vecs = embedder.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=normalize,
        show_progress_bar=False,
    )
    return np.array(vecs)

def topic_vecs(text):
    sents = sent_split(text)
    if not sents:
        return []
    if len(sents) < 2:
        return [encode(" ".join(sents)).mean(axis=0)]
    emb = encode(sents)
    k = min(max(1, len(sents)//8), 8)
    labels = AgglomerativeClustering(n_clusters=k).fit_predict(emb)
    segs = [" ".join([s for s, l in zip(sents, labels) if l == lab]) for lab in sorted(set(labels))]
    out = []
    for seg in segs:
        out.append(encode(seg).mean(axis=0))
    return out

def match_topics(vec, anchors_dict, max_topics=5, threshold=0.4):
    scores = {
        label: float(cosine_similarity([vec], [anchor])[0][0])
        for label, anchor in anchors_dict.items()
    }
    ranked = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    selected = []
    for i, (label, sim) in enumerate(ranked[:max_topics]):
        if i == 0 or sim >= threshold:
            selected.append({"topic_label": label, "similarity": sim})
    if not selected:
        selected = [{"topic_label": "General / Miscellaneous", "similarity": 0.0}]
    return selected

# --- Generate base topic embeddings + topic matches ---
topic_vec_list = topic_vecs(article_text)
topic_vecs_array = np.vstack(topic_vec_list)
print(f"Generated {len(topic_vecs_array)} base topic embeddings with shape {topic_vecs_array.shape}")

# --- Match to anchors ---
all_labels = []
topic_matches = []
for i, vec in enumerate(topic_vecs_array):
    matches = match_topics(vec, topic_anchors, max_topics=max_topics_per_vec, threshold=threshold)
    topic_matches.append(matches)
    top_labels = [m["topic_label"] for m in matches]
    all_labels.extend(top_labels)
    print(f"\n[Topic vector {i}] matches:")
    for m in matches:
        print(f"  - {m['topic_label']:<40} (similarity {m['similarity']:.3f})")

# --- Deduplicate + limit to top 8 overall topics ---
flat_topics = list(dict.fromkeys(all_labels))[:8]
print("\n--- Canonical Topics Assigned ---")
for t in flat_topics:
    print(f" - {t}")

# ================================================================
# NEW: Generate GPT summary for fine-grained topic matching
# ================================================================
print("\n--- Generating GPT Topic Summary ---")

summary_prompt = f"""
Summarize this article's specific subject in ONE sentence (max 20 words).
Be concrete and specific about what event, person, location, or issue is discussed.
Focus on the WHO/WHAT/WHERE, not opinion or analysis.

Article title: {meta.get('filename', 'Unknown')}
Text excerpt: {article_text[:2500]}

Example good summaries:
- "Discusses Israel's military operations in Gaza and humanitarian impact"
- "Analyzes Federal Reserve's interest rate decision and inflation outlook"
- "Covers Elon Musk's acquisition of Twitter and content moderation changes"
- "Reports on Supreme Court ruling on affirmative action in college admissions"

Summary:"""

try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": summary_prompt}],
        max_tokens=40,
        temperature=0.3
    )
    topic_summary = response.choices[0].message.content.strip()
    # Clean up any quotes or extra formatting
    topic_summary = topic_summary.strip('"').strip("'").strip()
    print(f"Topic summary: {topic_summary}")
except Exception as e:
    print(f"Warning: GPT summary failed ({e}), using title as fallback")
    topic_summary = meta.get('filename', 'Unknown article')

# Embed the summary
summary_vec = encode(topic_summary)  # shape (1, 768)

# Flatten to 1D if needed
if summary_vec.ndim == 2:
    summary_vec = summary_vec.flatten()  # Now shape (768,)

print(f"Summary embedding shape: {summary_vec.shape}")

# ================================================================
# Create composite topic vector: [base_topic_embedding | summary_embedding]
# ================================================================
# Use the first (primary) base topic vector
primary_topic_vec = topic_vecs_array[0]  # shape (768,)

# Ensure both are 1D
if primary_topic_vec.ndim == 2:
    primary_topic_vec = primary_topic_vec.flatten()

# Concatenate: base topic (768) + summary (768) = composite (1536)
composite_topic_vec = np.concatenate([primary_topic_vec, summary_vec])
print(f"\nComposite topic vector shape: {composite_topic_vec.shape}")
print(f"  - Base topic portion: indices 0-767")
print(f"  - Summary portion: indices 768-1535")

# --- Save all outputs ---
base = Path(meta["filename"]).stem

# Save composite vector (for retrieval)
composite_path = EPHEMERAL / f"{base}_topic_composite.npy"
np.save(composite_path, composite_topic_vec)

# Save base topic vectors (for backward compatibility/debugging)
topic_path = EPHEMERAL / f"{base}_topic.npy"
np.save(topic_path, topic_vecs_array)

# Save metadata
match_path = EPHEMERAL / f"{base}_topic_matches.json"
flat_path = EPHEMERAL / f"{base}_topics_flat.json"
summary_path = EPHEMERAL / f"{base}_topic_summary.json"

json.dump(topic_matches, open(match_path, "w"), indent=2)
json.dump(flat_topics, open(flat_path, "w"), indent=2)
json.dump({
    "summary": topic_summary,
    "embedding_version": "v2_composite",
    "composite_shape": list(composite_topic_vec.shape),
    "base_shape": list(primary_topic_vec.shape),
    "summary_shape": list(summary_vec.shape)
}, open(summary_path, "w"), indent=2)

print(f"\n--- Saved Artifacts ---")
print(f"Composite topic vector → {composite_path}")
print(f"Base topic vectors → {topic_path}")
print(f"Topic matches → {match_path}")
print(f"Canonical topics → {flat_path}")
print(f"Topic summary metadata → {summary_path}")


Embedding topics for: NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press (1).pdf (8738 chars)
Using topic model: intfloat/e5-base-v2
Loaded 22 topic anchors
Generated 5 base topic embeddings with shape (5, 768)

[Topic vector 0] matches:
  - Technology / Social Media & Platforms    (similarity 0.842)
  - Society / Media / Communication          (similarity 0.840)
  - Politics / US / Federal / Elections and Campaigns (similarity 0.835)
  - Technology / Innovation / AI             (similarity 0.829)
  - Ethics / Governance & Accountability / Integrity (similarity 0.823)

[Topic vector 1] matches:
  - Society / Culture & Identity / Social Issues (similarity 0.796)
  - Politics / Global / Geopolitics & Conflict (similarity 0.795)
  - Society / Media / Communication          (similarity 0.792)
  - Technology / Social Media & Platforms    (similarity 0.788)
  - Ethics / Governance & Accountability / Integrity (similarity 0.788)

[Topic vector 2] matches:
 

In [13]:
# ================================================================
# Stage 5b — Stance Classification + Hybrid Embedding (Ephemeral, Scraper-Accurate)
# ================================================================

import os, json, re, torch, numpy as np
from pathlib import Path
from openai import OpenAI
from sentence_transformers import SentenceTransformer

PROJECT_ROOT = Path("/content/unstructured-project").resolve()
CONFIG_DIR   = PROJECT_ROOT / "config"
TMP          = PROJECT_ROOT / "tmp"
EPHEMERAL    = TMP / "ephemeral_embeddings"
EPHEMERAL.mkdir(parents=True, exist_ok=True)

# --- Load configs and guides ---
with open(CONFIG_DIR / "political_leanings.json", encoding="utf-8") as f:
    leanings_map = json.load(f)
with open(CONFIG_DIR / "implied_stances.json", encoding="utf-8") as f:
    stances_map = json.load(f)
with open(CONFIG_DIR / "source_bias.json", encoding="utf-8") as f:
    source_bias = json.load(f)

# --- Load metadata and article text (FIXED to use explicit reference from Stage 4) ---
latest_meta_ref = TMP / "latest_meta.txt"
if not latest_meta_ref.exists():
    raise FileNotFoundError("Missing latest_meta.txt — run Stage 4 first to upload an article.")

latest_meta = Path(latest_meta_ref.read_text().strip())
if not latest_meta.exists():
    raise FileNotFoundError(f"Referenced meta file not found: {latest_meta}")

meta = json.load(open(latest_meta))
text_path = TMP / f"{Path(latest_meta).stem.replace('_meta','')}.txt"
if not text_path.exists():
    raise FileNotFoundError(f"Text file not found for: {text_path}")

article_txt = text_path.read_text(encoding="utf-8")
print(f"Generating stance embedding for: {meta['filename']} ({len(article_txt)} chars)")

# --- Retrieve outlet bias already inferred in Stage 4 ---
bias_family = meta.get("bias_family", "unknown")
bias_score  = float(meta.get("bias_score", 0.0))

# --- OpenAI client ---
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"].strip():
    from getpass import getpass
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# --- GPT classification identical to scraper ---
prompt = f"""
You are a political analyst.
Based on the article below, classify its overall political leaning (tone) and implied stance.

Leaning options: {', '.join(leanings_map.keys())}
Stance examples: {', '.join([s for cat in stances_map.values() for s in cat['families'].keys()])}

Return strict JSON with fields:
- political_leaning (string)
- implied_stance (string)
- summary (one-sentence summary of the article's main argument)

Article title: {meta.get('filename')}
Excerpt: {article_txt[:2000]}
"""

resp = client.chat.completions.create(
    model=CONFIG["stance_processing"]["llm"]["model"],
    messages=[{"role": "user", "content": prompt}],
    max_tokens=256,
    temperature=0.4
)
raw = resp.choices[0].message.content.strip()

try:
    stance_info = json.loads(raw)
except Exception:
    # fallback regex extraction
    leaning = re.search(r"leaning[:\-]?\s*(.+)", raw, re.I)
    stance  = re.search(r"stance[:\-]?\s*(.+)",  raw, re.I)
    summary = re.search(r"summary[:\-]?\s*(.+)", raw, re.I)
    stance_info = {
        "political_leaning": (leaning.group(1).strip() if leaning else "unknown"),
        "implied_stance":    (stance.group(1).strip()  if stance  else "unknown"),
        "summary":           (summary.group(1).strip() if summary else raw[:200])
    }

print("\n--- GPT Classification ---")
print(json.dumps(stance_info, indent=2))

# --- Compute tone score + match with outlet bias ---
def bias_to_score(label):
    l = (label or "").lower().strip()
    if "progressive" in l or ("left" in l and "center" not in l): return -0.8
    if "center left" in l:  return -0.4
    if l == "center":       return 0.0
    if "center right" in l: return 0.4
    if "conservative" in l or "right" in l: return 0.8
    if "libertarian" in l:  return 0.6
    return 0.0

tone_score = bias_to_score(stance_info.get("political_leaning"))
author_match = abs(bias_score - tone_score) <= 0.3

# --- Build enriched stance metadata ---
stance_meta = {
    "political_leaning": stance_info.get("political_leaning", "unknown"),
    "implied_stance":    stance_info.get("implied_stance", "unknown"),
    "summary":           stance_info.get("summary", ""),
    "bias_family":       bias_family,
    "bias_score":        bias_score,
    "tone_score":        tone_score,
    "author_tone_match": author_match
}

print("\n--- Source / Tone Alignment ---")
print(json.dumps(stance_meta, indent=2))

# --- Hybrid text for embedding ---
hybrid_text = "\n".join([
    stance_meta["political_leaning"],
    stance_meta["implied_stance"],
    stance_meta["summary"]
]).strip()

# --- Generate embedding ---
stance_model_name = CONFIG["embeddings"]["stance_model"]
device = "cuda" if torch.cuda.is_available() else "cpu"
embedder = SentenceTransformer(stance_model_name, device=device)
stance_vec = embedder.encode(hybrid_text, normalize_embeddings=True)
stance_vec = stance_vec.reshape(1, -1)
print(f"\nUsing stance model: {stance_model_name}")
print(f"Generated stance vector with shape {stance_vec.shape}")

# --- Save ephemeral outputs for Stage 6 ---
base = Path(meta["filename"]).stem
np.save(EPHEMERAL / f"{base}_stance.npy", stance_vec)
Path(EPHEMERAL / f"{base}_stance_summary.txt").write_text(hybrid_text, encoding="utf-8")
Path(EPHEMERAL / f"{base}_stance_info.json").write_text(json.dumps(stance_meta, indent=2), encoding="utf-8")

print(f"\nSaved ephemeral stance artifacts under {EPHEMERAL}")


Generating stance embedding for: NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press (1).pdf (8738 chars)

--- GPT Classification ---
{
  "political_leaning": "\": \"center left\",",
  "implied_stance": "\": \"anti elite populism\",",
  "summary": "\": \"The article discusses the concerns of some Jewish voters regarding Zohran Mamdani's views, particularly on Israel, while acknowledging that he does not exhibit antisemitism in the traditional sense.\""
}

--- Source / Tone Alignment ---
{
  "political_leaning": "\": \"center left\",",
  "implied_stance": "\": \"anti elite populism\",",
  "summary": "\": \"The article discusses the concerns of some Jewish voters regarding Zohran Mamdani's views, particularly on Israel, while acknowledging that he does not exhibit antisemitism in the traditional sense.\"",
  "bias_family": "progressive left",
  "bias_score": -0.7,
  "tone_score": -0.4,
  "author_tone_match": true
}

Using stance model: all-mpnet-base-v2

### Stage 6 — Retrieval and Anti-Echo Analysis

This stage retrieves articles related to the uploaded article and groups them by stance similarity and topic relevance.

---

### Process Summary

* **Load uploaded article features**:

  * Topic embedding (composite of topic + summary)
  * Bias and tone scores (from stance metadata)
* **Retrieve corpus candidates** from Chroma using topic similarity.
* **Compute comparison metrics** for each candidate:

  * **summary_similarity** – cosine similarity between summary/topic embeddings
  * **canonical_topic_overlap** – Jaccard overlap of canonical topic tags
  * **stance_similarity** – cosine similarity between stance embeddings
  * **bias_diff** – absolute difference in bias scores
  * **tone_diff** – absolute difference in tone scores

---

### Ranking Logic

Articles are grouped and ranked in tiers:

| Category                        | Criteria                   | Sort Key                                      |
| ------------------------------- | -------------------------- | --------------------------------------------- |
| **Same topic, similar stance**  | High summary similarity    | summary_similarity ↓                          |
| **Same topic, opposite stance** | High topic relevance       | stance_similarity ↑ then summary_similarity ↓ |
| **Different source bias**       | Bias significantly differs | bias_diff ↓ then summary_similarity ↓         |
| **Mixed/neutral**               | Sorted by hybrid signal    | anti_echo_score ↓                             |

---

### Scoring Formula

```
anti_echo_score =
    (w_topic  * summary_similarity)
  - (w_stance * stance_similarity)
  + (w_bias   * bias_diff)
  + (w_tone   * tone_diff)
```

---

### Output

* **Printed sections**:

  * Same topic, similar stance
  * Same topic, opposite stance
  * Different source bias
  * Full ranked list
* **CSV export** of metrics:

  * summary_similarity
  * canonical_topic_overlap
  * stance_similarity
  * bias_diff
  * tone_diff
  * anti_echo_score

In [18]:
# ================================================================
# Stage 6 — Retrieval and Anti-Echo Analysis (Composite Topic + Stance)
# ================================================================

import os, json, numpy as np, pandas as pd
from pathlib import Path
from openai import OpenAI
import chromadb
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from getpass import getpass
from itertools import product

# ---------------------------------------------------------------
# TUNABLE PARAMETERS
# ---------------------------------------------------------------
w_T_canonical = 0.5
w_T_summary = 10.0
w_S = 1.0
w_B = 0.8
w_Tone = 0.3

CANONICAL_TOPIC_THRESHOLD = 0.3
SUMMARY_SIMILARITY_THRESHOLD = 0.8
TOP_N_RESULTS = 10
PRINT_TOP_N = 3

# ---------------------------------------------------------------
# Configuration and paths
# ---------------------------------------------------------------
PROJECT_ROOT = Path("/content/unstructured-project").resolve()
CHROMA_PATH = PROJECT_ROOT / "chroma_db"
TMP = PROJECT_ROOT / "tmp"
EPHEMERAL = TMP / "ephemeral_embeddings"

# ---------------------------------------------------------------
# OpenAI setup
# ---------------------------------------------------------------
if "OPENAI_API_KEY" not in os.environ or not os.environ["OPENAI_API_KEY"].strip():
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# ---------------------------------------------------------------
# Load metadata and ephemeral embeddings (FIXED to use Stage 4 pointer)
# ---------------------------------------------------------------
latest_meta_ref = TMP / "latest_meta.txt"
if not latest_meta_ref.exists():
    raise FileNotFoundError("Missing latest_meta.txt — run Stage 4 first to upload an article.")

latest_meta = Path(latest_meta_ref.read_text().strip())
if not latest_meta.exists():
    raise FileNotFoundError(f"Referenced meta file not found: {latest_meta}")

meta = json.load(open(latest_meta))
article_id = Path(meta["filename"]).stem
bias_score_article = float(meta["bias_score"])
tone_score_article = float(meta.get("tone_score", 0.0))
print(f"Running anti-echo retrieval for {meta['filename']} (bias={bias_score_article}, tone={tone_score_article})")

composite_topic_vec = np.load(EPHEMERAL / f"{article_id}_topic_composite.npy")
uploaded_base_topic = composite_topic_vec[:768]
uploaded_summary = composite_topic_vec[768:]

stance_vec = np.load(EPHEMERAL / f"{article_id}_stance.npy")
topics_flat = json.load(open(EPHEMERAL / f"{article_id}_topics_flat.json"))
topic_summary_meta = json.load(open(EPHEMERAL / f"{article_id}_topic_summary.json"))
topic_summary = topic_summary_meta.get("summary", "")

print(f"\nUploaded article topic summary: {topic_summary}")
print(f"Composite vector shape: {composite_topic_vec.shape}")
print(f"  Base topic: {uploaded_base_topic.shape}")
print(f"  Summary: {uploaded_summary.shape}")

client_chroma = chromadb.PersistentClient(path=str(CHROMA_PATH))
topic_coll = client_chroma.get_collection(CONFIG["chroma_collections"]["topic"])
stance_coll = client_chroma.get_collection(CONFIG["chroma_collections"]["stance"])

# ---------------------------------------------------------------
# Helpers
# ---------------------------------------------------------------
def parse_topics(obj):
    if obj is None:
        return []
    if isinstance(obj, list):
        return [t.strip() for t in obj if t.strip()]
    if isinstance(obj, str):
        parts = [t.strip() for t in obj.split(";") if t.strip()]
        if len(parts) == 1 and parts[0].startswith("["):
            try:
                parsed = json.loads(parts[0])
                if isinstance(parsed, list):
                    return [t.strip() for t in parsed if isinstance(t, str)]
            except Exception:
                pass
        return parts
    return []

def topic_overlap_score_multi(upload_topics, candidate_topics, anchors_dict=None):
    A = set([t.strip().lower() for t in parse_topics(upload_topics)])
    B = set([t.strip().lower() for t in parse_topics(candidate_topics)])
    if not A or not B:
        return 0.0
    jaccard = len(A & B) / len(A | B)
    if anchors_dict:
        sims = []
        for a, b in product(A, B):
            if a in anchors_dict and b in anchors_dict:
                va, vb = anchors_dict[a], anchors_dict[b]
                sim = float(cosine_similarity([va], [vb])[0][0])
                sims.append(sim)
        if sims:
            return max(jaccard, max(sims))
    return jaccard

def interpret_bias(score: float) -> str:
    if score <= -0.6: return "Progressive / Left"
    if -0.6 < score <= -0.2: return "Center-Left"
    if -0.2 < score < 0.2: return "Center / Neutral"
    if 0.2 <= score < 0.6: return "Center-Right"
    if score >= 0.6: return "Conservative / Right"
    return "Unknown"

# ---------------------------------------------------------------
# Retrieve all topic & stance docs
# ---------------------------------------------------------------
topic_docs = topic_coll.get(include=["embeddings", "metadatas"])
stance_docs = stance_coll.get(include=["embeddings", "metadatas"])
unique_articles_in_db = len(set(md.get("id", "").split("::")[0] for md in topic_docs["metadatas"]))

print(f"\nRetrieved from Chroma:")
print(f"  - {len(topic_docs['ids'])} topic vectors")
print(f"  - {len(stance_docs['ids'])} stance vectors")
print(f"  - {unique_articles_in_db} unique articles")

# ---------------------------------------------------------------
# Compare uploaded article to stored corpus
# ---------------------------------------------------------------
print(f"\nScanning corpus for matches...")
anchors_path = PROJECT_ROOT / "config/topic_anchors.npz"
anchors_npz = np.load(anchors_path, allow_pickle=True)
topic_anchors = {k: anchors_npz[k] for k in anchors_npz.files}

topic_model_name = CONFIG["embeddings"]["topic_model"]
device = "cuda" if torch.cuda.is_available() else "cpu"
topic_embedder = SentenceTransformer(topic_model_name, device=device)

def encode_text(text):
    vec = topic_embedder.encode(text, normalize_embeddings=True, show_progress_bar=False)
    if vec.ndim == 2:
        vec = vec.flatten()
    return vec

all_matches = []
for emb, md in zip(topic_docs["embeddings"], topic_docs["metadatas"]):
    canonical_overlap = topic_overlap_score_multi(topics_flat, md.get("topics_flat", []), anchors_dict=topic_anchors)
    if canonical_overlap < CANONICAL_TOPIC_THRESHOLD:
        continue

    emb_array = np.array(emb)
    if len(emb_array) == 1536:
        candidate_summary_vec = emb_array[768:]
        summary_similarity = cosine_similarity(uploaded_summary.reshape(1, -1), candidate_summary_vec.reshape(1, -1))[0][0]
        old_summary = "(new format)"
    else:
        old_summary = md.get("openai_topic_summary", "")
        if old_summary:
            candidate_summary_vec = encode_text(old_summary)
            summary_similarity = cosine_similarity(uploaded_summary.reshape(1, -1), candidate_summary_vec.reshape(1, -1))[0][0]
        else:
            summary_similarity = cosine_similarity(uploaded_base_topic.reshape(1, -1), emb_array.reshape(1, -1))[0][0]

    if summary_similarity < SUMMARY_SIMILARITY_THRESHOLD:
        continue

    article_id_base = md.get("id", "").split("::")[0]
    all_matches.append({
        "article_id": article_id_base,
        "row_id": md.get("id"),
        "source": md.get("source", ""),
        "title": md.get("title", ""),
        "url": md.get("url", ""),
        "bias_family": md.get("bias_family", ""),
        "topics_flat": md.get("topics_flat", []),
        "canonical_overlap": canonical_overlap,
        "summary_similarity": summary_similarity,
        "candidate_summary": old_summary,
        "metadata": md
    })

print(f"Found {len(all_matches)} topic vector matches from {len(set(m['article_id'] for m in all_matches))} unique articles")

best_matches = {}
for match in all_matches:
    aid = match["article_id"]
    if aid not in best_matches or match["summary_similarity"] > best_matches[aid]["summary_similarity"]:
        best_matches[aid] = match

print(f"After deduplication: {len(best_matches)} unique articles")

# ---------------------------------------------------------------
# Stance, bias, and final scoring
# ---------------------------------------------------------------
scores = []
for aid, match in best_matches.items():
    md = match["metadata"]

    bias_db, tone_db = 0.0, 0.0
    for s_md in stance_docs["metadatas"]:
        s_aid = s_md.get("id", "").split("::")[0]
        if s_aid == aid:
            try:
                bias_db = float(s_md.get("bias_score", 0.0))
            except Exception:
                try:
                    bias_db = float(json.loads(s_md.get("source_bias", "{}")).get("bias_score", 0.0))
                except Exception:
                    bias_db = 0.0
            tone_db = float(s_md.get("tone_score", 0.0))
            break

    bias_diff = abs(bias_score_article - bias_db)
    tone_diff = abs(tone_score_article - tone_db)

    stance_match = None
    for s_emb, s_md in zip(stance_docs["embeddings"], stance_docs["metadatas"]):
        if s_md.get("id", "").split("::")[0] == aid:
            stance_match = s_emb
            break

    stance_sim = 0.0
    if stance_match is not None:
        stance_sim = cosine_similarity(stance_vec.reshape(1, -1), np.array(stance_match).reshape(1, -1))[0][0]

    anti_echo_score = (
        (w_T_canonical * match["canonical_overlap"]) +
        (w_T_summary * match["summary_similarity"]) -
        (w_S * stance_sim) -
        (w_B * bias_diff) -
        (w_Tone * tone_diff)
    )

    scores.append({
        "article_id": aid,
        "source": match["source"],
        "title": match["title"],
        "url": match["url"],
        "bias_family": match["bias_family"],
        "bias_score": bias_db,
        "canonical_topic_overlap": match["canonical_overlap"],
        "summary_similarity": match["summary_similarity"],
        "stance_similarity": stance_sim,
        "bias_diff": bias_diff,
        "tone_diff": tone_diff,
        "anti_echo_score": anti_echo_score
    })

df = pd.DataFrame(scores)
if df.empty:
    print("\n" + "="*80)
    print("WARNING: No related articles found")
    print("="*80)
    print("This could mean:")
    print("1. No articles in the database share similar topics")
    print("2. Thresholds are too strict:")
    print(f"   - Canonical topic threshold: {CANONICAL_TOPIC_THRESHOLD}")
    print(f"   - Summary similarity threshold: {SUMMARY_SIMILARITY_THRESHOLD}")
    print("\nTry lowering thresholds or checking your canonical topics:")
    print(f"Your topics: {topics_flat}")
    print("="*80)
    raise ValueError("No related articles found")

print(f"\nFinal result: {len(df)} unique articles matched")

# ---------------------------------------------------------------
# Ranking
# ---------------------------------------------------------------
same_topic_diff_bias = df.sort_values(["bias_diff", "summary_similarity"], ascending=[False, False]).head(TOP_N_RESULTS)

same_topic_opposite_stance = df[
    (df["summary_similarity"] >= SUMMARY_SIMILARITY_THRESHOLD) &
    (df["canonical_topic_overlap"] >= CANONICAL_TOPIC_THRESHOLD)
].sort_values(["stance_similarity", "summary_similarity"], ascending=[True, False]).head(TOP_N_RESULTS)

anti_echo_best = df.sort_values("anti_echo_score", ascending=False).head(TOP_N_RESULTS)

# ---------------------------------------------------------------
# Output helpers (FULL URL VERSION)
# ---------------------------------------------------------------
def print_header(title):
    print("\n" + "=" * 80)
    print(title.upper().center(80))
    print("=" * 80 + "\n")

def format_article_row(row):
    title = (row.get("title") or "Untitled").strip()
    source = row.get("source", "unknown")
    bias_label = interpret_bias(row["bias_score"])
    metrics = (
        f"Canonical overlap: {row['canonical_topic_overlap']:.2f}   "
        f"Summary sim: {row['summary_similarity']:.2f}   "
        f"Stance sim: {row['stance_similarity']:.2f}   "
        f"Bias diff: {row['bias_diff']:.2f}"
    )
    lines = [
        f"• {title}",
        f"  Source: {source}  ({bias_label})",
        f"  {metrics}",
        f"  Anti-echo score: {row['anti_echo_score']:.3f}",
    ]
    url = row.get("url", "")
    if url:
        lines.append(f"  Link: {url}")  # full URL (no truncation)
    return "\n".join(lines)

def show_results(df, title, n=PRINT_TOP_N):
    print_header(title)
    if df.empty:
        print("  No matches found.\n")
        return
    for _, row in df.head(n).iterrows():
        print(format_article_row(row))
        print("-" * 80)
    print()

def show_overview(df):
    print_header("Ideological Spread Overview")
    left = df[df["bias_score"] < -0.2]["source"].unique()
    right = df[df["bias_score"] > 0.2]["source"].unique()
    print(f"Left / progressive outlets : {', '.join(left) if len(left)>0 else 'none'}")
    print(f"Right / conservative outlets: {', '.join(right) if len(right)>0 else 'none'}\n")
    print(f"Search parameters:")
    print(f"  Your article summary: {topic_summary}")
    print(f"  Canonical topics: {', '.join(topics_flat[:3])}{'...' if len(topics_flat) > 3 else ''}\n")
    top = df.iloc[0]
    print(f"Top matching article: {top['source']} ({interpret_bias(top['bias_score'])})")
    print(f"  Title: {(top['title'] or 'Untitled')[:60]}...")
    print(f"  Canonical overlap: {top['canonical_topic_overlap']:.2f}   "
          f"Summary sim: {top['summary_similarity']:.2f}   "
          f"Stance sim: {top['stance_similarity']:.2f}   "
          f"Bias diff: {top['bias_diff']:.2f}")
    print(f"  Anti-echo score: {top['anti_echo_score']:.3f}\n")

# ---------------------------------------------------------------
# Display results
# ---------------------------------------------------------------
show_overview(anti_echo_best)
show_results(same_topic_diff_bias, "Same Specific Topic — Different Source Bias")
show_results(same_topic_opposite_stance, "Same Specific Topic — Opposite Stance")
show_results(anti_echo_best, "Top Anti-Echo Candidates (Best Overall Matches)")

# ---------------------------------------------------------------
# Save results
# ---------------------------------------------------------------
out_path = TMP / f"{article_id}_anti_echo_analysis.csv"
df.to_csv(out_path, index=False)
print("=" * 80)
print(f"Detailed analysis saved to: {out_path}")
print(f"Total matches found: {len(df)}")
print("=" * 80)


Running anti-echo retrieval for NYT_Reprint_Opinion_ Why Mamdani frightens Jews like me _ Chattanooga Times Free Press (1).pdf (bias=-0.7, tone=0.0)

Uploaded article topic summary: Examines Jewish voters' concerns about mayoral candidate Zohran Mamdani's anti-Zionist views in New York City.
Composite vector shape: (1536,)
  Base topic: (768,)
  Summary: (768,)

Retrieved from Chroma:
  - 2168 topic vectors
  - 432 stance vectors
  - 432 unique articles

Scanning corpus for matches...
Found 61 topic vector matches from 14 unique articles
After deduplication: 14 unique articles

Final result: 14 unique articles matched

                          IDEOLOGICAL SPREAD OVERVIEW                           

Left / progressive outlets : guardian
Right / conservative outlets: cityjournal, foxnews

Search parameters:
  Your article summary: Examines Jewish voters' concerns about mayoral candidate Zohran Mamdani's anti-Zionist views in New York City.
  Canonical topics: Technology / Social Media &